In [116]:
from spacy.tokens import Doc, Span
from datasets import Dataset
from typing import List

def bigbio_to_spacy(nlp, bigbio_dataset : Dataset, is_sentencized : bool = False) -> List[Doc]:
    result = []
    for d in bigbio_dataset:
        doc_text = " ".join([t for p in d["passages"] for t in p["text"]])
        doc = nlp(doc_text)
        spans = []
        for e in d['entities']:
            span = doc.char_span(e['offsets'][0][0], e['offsets'][-1][1], label=e['type'])
            spans.append(span)
        doc.spans['snomed'] = spans
        doc.user_data['document_id'] = d['document_id']
        if is_sentencized:
            doc[0].is_sent_start = True
            for t in doc[1:]:
                t.is_sent_start = False
        result.append(doc)
    return result

In [117]:
from datasets import load_dataset

ggponc_ds = load_dataset('bigbio/ggponc2', data_dir='data/v2.0_2022_03_24', name='ggponc2_fine_long_bigbio_kb')

In [119]:
nlp = spacy.blank('de')
nlp.add_pipe('sentencizer')

spacy_test = bigbio_to_spacy(nlp, ggponc_ds['test'])

In [1]:
import spacy
from spacy.tokens import DocBin
from itertools import islice

In [112]:
nlp = spacy.blank('de')
db = DocBin(store_user_data=True).from_disk('data/v2.0_2022_03_24/annotations/spacy/test.spacy')
docs = list(db.get_docs(nlp.vocab))
for d in docs:
    d[0].is_sent_start = True
    for t in d[1:]:
        t.is_sent_start = False
docs[0:5]

[Die Panorama-Schichtaufnahme gehört zur zahnärztlichen Basisdiagnostik und sollte vor Beginn der spezifischen Tumortherapie vorliegen . ,
 Bei zu erwartenden Metallartefakten im Bereich der Mundhöhle sollte die MRT dem CT zur Beurteilung des Primärtumors vorgezogen werden . ,
 Die diagnostische Spezifität des Lymphknotenstagings am Hals kann durch die Ultraschall gestützte Feinnadelbiopsie verbessert werden . ,
 Um einen positiven Resektionsrand , der mit einer schlechteren Prognose verbunden ist zu vermeiden , kann die Technik einer intraoperativen Schnellschnitthistologie hilfreich sein . ,
 Der histopathologische Befund soll in Kommunikation mit dem Kliniker die genaue Lokalisation einer ggf . vorliegenden R + - Situation beschreiben . ]

In [114]:
len(docs)

13714

In [115]:
len(spacy_test)

1529

In [93]:
d1 = from_spacy(docs, span_key='snomed')

In [120]:
d2 = from_spacy(spacy_test, span_key='snomed')

In [121]:
from xmen.evaluation import evaluate
from xmen.data import from_spacy

evaluate(d2, d2, ner_only=True)

{'strict': {'precision': 1.0,
  'recall': 1.0,
  'fscore': 1.0,
  'ptp': 34162,
  'fp': 0,
  'rtp': 34162,
  'fn': 0,
  'n_docs_system': 1529,
  'n_annos_system': 34414,
  'n_docs_gold': 1529,
  'n_annos_gold': 34414}}

In [68]:
ents_spacy = []
for d in conv_test:
    for s in d.spans['snomed']:
        ents_spacy.append(s)

In [69]:
len(ents_spacy)

34414

In [70]:
ents_spacy[123:127]

[Radiochemotherapie,
 Behandlung des Mundhöhlenkarzinoms,
 Mundhöhlenkarzinoms,
 Strahlentherapie]